# Adaptive integration

In [ ]:
using FundamentalsNumericalComputation

In [ ]:
f = x -> (x+1)^2*cos((2*x+1)/(x-4.3));

@show exact,errest = quadgk(f,0,4,atol=1e-14,rtol=1e-14);  # 'exact' value

We perform the integration and show the nodes selected underneath the curve.

In [ ]:
Q,t = FNC.intadapt(f,0,4,0.001)
@show num_nodes = length(t);

plot(f,0,4,color=:black,legend=:none,
    xlabel="x", ylabel="f(x)", title="Adaptive node selection")
plot!(t,f.(t),seriestype=:sticks,m=(:o,2))

The error turns out to be a bit more than we requested. It's only an estimate, not a guarantee.

In [ ]:
@show err = exact - Q;

Let's see how the number of integrand evaluations and the error vary with the requested tolerance.

In [ ]:
table = (tol=[],err=[],n=[])
for tol in 10.0.^(-4:-1:-14)
    Q,t = FNC.intadapt(f,0,4,tol)
    push!(table.tol,tol)
    push!(table.err,exact-Q)
    push!(table.n,length(t))
end

pretty_table(table,["tolerance","error","f-evaluations"],backend=:html)

As you can see, even though the errors are not less than the estimates, the two columns decrease in tandem. If we consider now the convergence not in $h$ (which is poorly defined) but in the number of nodes actually chosen, we come close to the fourth order accuracy of the underlying Simpson scheme.

In [ ]:
n = table.n
plot(n,abs.(table.err),m=:o,label="results",
    xaxis=(:log10,"number of nodes"), yaxis=(:log10,"error"),
    title="Convergence of adaptive quadrature")

order4 = @. 0.01*(n/n[1])^(-4)
plot!(n,order4,l=:dash,label="\$O(n^{-4})\$")